In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# Decision tree

# Creating data

In [21]:
data = [
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon'],
]

# column label
header = ['Color','Diameter','Label']

In [22]:
def unique(rows,col):
    return set([row[col] for row in rows])

# count number of each type of example in dataset
def class_counts(rows):
    count={}
    for row in rows:
        label=row[-1]
        if label not in count:
            count[label]=0
        count[label] += 1
    return count

def isnumeric(value):
    return isinstance(value,int) or isinstance(value,float)


# Partition dataset 

In [34]:
class Question:
    
    def __init__(self,column,value):
        self.column=column
        self.value=value
    
    def match(self,example):
        val = example[self.column]
        
        if isnumeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __ref__(self):
        condition = "=="
        if isnumeric(self.value):
            condition = ">="
            
        return "Is %s %s %s?"%(
            header[self.column],condition,str(self.value)
        )
        
        

In [32]:
def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows


def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_lbl**2
    return impurity


def info_gain(left,right,current_uncertanity):
    p = float(len(left))/(len(left)+len(right))
    return current_uncertanity - p*gini(left) - (1-p)*gini(right)


def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertanity = gini(rows)
    n_features = len(rows[0])-1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            question = Question(col,val)
            true_rows,false_rows = partition(rows,question)
            
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            
            gain = info_gain(true_rows,false_rows,current_uncertanity)
            
            if gain >= best_gain:
                best_gain,best_question = gain,question
                
                
    return best_gain,best_question 
        

In [28]:
# leaf node
class leaf:
    
    def __init__(self,rows):
        self.predictions = class_counts(rows)

In [36]:
class decision_node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
        
def build_tree(rows):
    
    gain,question = find_best_split(rows)
    
    if gain==0:
        return leaf(rows)
    
    true_rows,false_rows = partition(rows,question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return decision_node(question,true_branch,false_branch)
    
def print_tree(node,spacing=""):
    
    if isinstance(node,leaf):
        print(spacing + "predict",  node.predictions)
        return
    
    print(spacing + str(node.question))
    
    print(spacing + '--> True:')
    print_tree(node.true_branch,spacing="  ")
    
    print(spacing + '--> False:')
    print_tree(node.false_branch,spacing="  ")
    

def classify(row,node):
    if isinstance(node,leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)
    
def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs = {}
    
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total*100)) + "%"
    return probs



if __name__ == '__main__':
    my_tree = build_tree(data)
    
    print_tree(my_tree)
    
    data = [
       ['Green',3,'Mango'],
       ['Yellow',3,'Mango'],
       ['Red',1,'Grape'],
       ['Red',1,'Grape'],
       ['Yellow',3,'Lemon'],
    ]
    
    for row in data:
        print("Actual: %s, predicted: %s"%(row[-1],print_leaf(classify(row,my_tree))))



--> True:
  --> True:
  predict {'Mango': 1, 'Lemon': 1}
  --> False:
  predict {'Mango': 1}
--> False:
  predict {'Grape': 2}
Actual: Mango, predicted: {'Mango': '100%'}
Actual: Mango, predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape, predicted: {'Grape': '100%'}
Actual: Grape, predicted: {'Grape': '100%'}
Actual: Lemon, predicted: {'Mango': '50%', 'Lemon': '50%'}
